In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [4]:
dataset=dataset.drop("User ID",axis=1)

In [5]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [6]:
dataset["Purchased"].value_counts()

0    257
1    143
Name: Purchased, dtype: int64

In [7]:
indep=dataset[["Age","EstimatedSalary","Gender_Male"]]
dep=dataset[["Purchased"]]

In [8]:
indep.shape

(400, 3)

In [9]:
dep

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [12]:
from sklearn.svm import SVC

In [13]:
from sklearn.model_selection import GridSearchCV
param_grid = {'kernel':['linear','rbf','poly','sigmoid'],'gamma':['auto','scale'],'C':[10,100,1000,2000,3000]}
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3,cv=5,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train, y_train)

C:\Users\sony\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 40 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done  72 out of 120 | elapsed:    7.4s remaining:    4.9s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    8.2s finished
C:\Users\sony\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\sony\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=-1,
             param_grid={'C': [10, 100, 1000, 2000, 3000],
                         'gamma': ['auto', 'scale'],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1_weighted', verbose=3)

In [14]:
re=grid.cv_results_
grid_predictions = grid.predict(X_test)

In [15]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

In [16]:
print("The Confusion Matrix:",cm)

The Confusion Matrix: [[77  8]
 [ 4 45]]


In [17]:
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [18]:
print("The Report:",clf_report)

The Report:               precision    recall  f1-score   support

           0       0.95      0.91      0.93        85
           1       0.85      0.92      0.88        49

    accuracy                           0.91       134
   macro avg       0.90      0.91      0.91       134
weighted avg       0.91      0.91      0.91       134



In [19]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

AttributeError: predict_proba is not available when  probability=False

In [20]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.009327,9.413380e-04,0.003997,9.602742e-07,10,auto,linear,"{'C': 10, 'gamma': 'auto', 'kernel': 'linear'}",0.771748,0.742857,0.917693,0.810473,0.076310,25
1,0.007328,9.419547e-04,0.004663,9.424036e-04,10,auto,rbf,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",0.888038,0.865289,0.931818,0.894996,0.027512,1
2,0.007995,1.632777e-03,0.004665,9.420102e-04,10,auto,poly,"{'C': 10, 'gamma': 'auto', 'kernel': 'poly'}",0.812816,0.802404,0.918637,0.844380,0.052385,11
3,0.007329,3.397339e-03,0.003998,1.632485e-03,10,auto,sigmoid,"{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.748923,0.704545,0.750000,0.734598,0.021135,37
4,0.007329,1.883852e-03,0.003331,9.420108e-04,10,scale,linear,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}",0.771748,0.742857,0.917693,0.810473,0.076310,25
5,0.007329,3.397869e-03,0.004663,9.422350e-04,10,scale,rbf,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",0.888038,0.865289,0.931818,0.894996,0.027512,1
6,0.007375,9.777090e-04,0.003332,1.884301e-03,10,scale,poly,"{'C': 10, 'gamma': 'scale', 'kernel': 'poly'}",0.812816,0.802404,0.918637,0.844380,0.052385,11
7,0.005332,9.432473e-04,0.003997,2.869840e-06,10,scale,sigmoid,"{'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid'}",0.753684,0.704545,0.745617,0.734759,0.021498,36
8,0.022651,5.728721e-03,0.004663,9.430225e-04,100,auto,linear,"{'C': 100, 'gamma': 'auto', 'kernel': 'linear'}",0.771748,0.742857,0.917693,0.810473,0.076310,25
9,0.008661,1.884021e-03,0.004663,9.426290e-04,100,auto,rbf,"{'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}",0.888038,0.876797,0.920148,0.894942,0.018310,3
